In [1]:
# !pip install https://github.com/explosion/spacy-models/releases//download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz

In [1]:
import wikipediaapi
import spacy
nlp = spacy.load('en_core_web_sm') #en_core_web_lg
# nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# import textacy

In [2]:
#from textacy.subject_verb_object_extract import findSVOs
from subject_verb_object_extract import findSVOs
from src_rel_tgt_extract import findSVAOs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
punct='!""#$%&\'()*+-/:;<=>?@[\\]^_”’“`{|}~``''—'  

from collections import Counter
import operator
from nltk.corpus import wordnet

l_stopwords = stopwords.words('english')


import py_stringmatching as sm
 
SW=sm.SmithWaterman()
GJ=sm.GeneralizedJaccard()
lev = sm.Levenshtein()
jac = sm.Jaccard()
ocf=sm.OverlapCoefficient()
 
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
 
from py_stringmatching.similarity_measure.partial_token_sort import PartialTokenSort
pr= PartialRatio()
pts= PartialTokenSort()

In [3]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm

In [4]:
reject_ent_list = [ 'CARDINAL', 'DATE','WORK_OF_ART','MONEY','ORDINAL','FAC','TIME','LOC','QUANTITY','LAW','PERCENT']

In [5]:
def rem_title(data):
    mod_list = []
    for x in data.split('\n\n'):
        word_count = len(x.split())
        if word_count > 15:
            # print(x, '->', word_count )
            mod_list.append(x)
    final_text = ''.join(mod_list)
    return final_text

### Wiki Pedia Data for chip shortage & semiconductors

In [6]:
def scrape_wikipedia(name_topic, verbose=True):
    def link_to_wikipedia(link):
        try:
            page = api_wikipedia.page(link)
            if page.exists():
                return {'page': link, 'text': page.text, 'link': page.fullurl, 'categories': list(page.categories.keys())}
        except:
            return None
      
    api_wikipedia = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    name_of_page = api_wikipedia.page(name_topic)
    if not name_of_page.exists():
        print('Page {} is not present'.format(name_of_page))
        return
  
    links_to_page = list(name_of_page.links.keys())
    procceed = tqdm(desc='Scraped links', unit='', total=len(links_to_page)) if verbose else None
    origin = [{'page': name_topic, 'text': name_of_page.text, 'link': name_of_page.fullurl, 'categories': list(name_of_page.categories.keys())}]
  
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        links_future = {executor.submit(link_to_wikipedia, link): link for link in links_to_page}
        for future in concurrent.futures.as_completed(links_future):
            info = future.result()
            origin.append(info) if info else None
            procceed.update(1) if verbose else None
    procceed.close() if verbose else None
  
    namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                 'Template', 'Help', 'User', 'Category talk', 'Portal talk')
    origin = pd.DataFrame(origin)
    origin = origin[(len(origin['text']) > 20)
                      & ~(origin['page'].str.startswith(namespaces, na=True))]
    origin['categories'] = origin.categories.apply(lambda a: [b[9:] for b in a])

    origin['topic'] = name_topic
    print('Scraped pages', len(origin))
  
    return origin

In [57]:
# data_chip_shortage = scrape_wikipedia('chip shortage')
# data_chip_act = scrape_wikipedia('CHIPS and Science Act')
# data_semi_supply = scrape_wikipedia('2021–2022 global supply chain crisis')

In [58]:
# data_chip_shortage.head()

In [59]:
# data_chip_act.head()

In [60]:
# data_semi_supply.head()

In [61]:
# data preparation to combine key words.
# df_combined_wiki = pd.concat([data_chip_shortage,data_chip_act,data_semi_supply])
# df_combined_wiki.to_csv('scraped_data_cs_all.csv',index=False)
# len(df_combined_wiki)

In [7]:
import spacy
import pandas as pd
import requests
from spacy import displacy
import nltk
# import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')

from spacy.tokens import Span
from spacy.matcher import Matcher

import matplotlib.pyplot as plot
from tqdm import tqdm
# import neptune.new as neptune

%matplotlib inline

In [8]:
wiki_data = pd.read_csv('scraped_data_cs_all.csv')
wiki_data.columns = ['page', 'news', 'url', 'categories', 'topic']
wiki_data.tail()

,page,news,url,categories,topic
373,COVID-19 pandemic in Belarus,The COVID-19 pandemic in Belarus is part of th...,https://en.wikipedia.org/wiki/COVID-19_pandemi...,"['2020 in Belarus', '2021 in Belarus', 'Articl...",2021–2022 global supply chain crisis
374,COVID-19 pandemic in Belgium,The COVID-19 pandemic in Belgium has resulted ...,https://en.wikipedia.org/wiki/COVID-19_pandemi...,"['2020 in Belgium', '2021 in Belgium', 'All Wi...",2021–2022 global supply chain crisis
375,COVID-19 pandemic in Bihar,The first COVID-19 case in the Indian state of...,https://en.wikipedia.org/wiki/COVID-19_pandemi...,['All Wikipedia articles written in Indian Eng...,2021–2022 global supply chain crisis
376,COVID-19 pandemic in Bhutan,The COVID-19 pandemic in Bhutan is part of the...,https://en.wikipedia.org/wiki/COVID-19_pandemi...,"['2020 in Bhutan', '2021 in Bhutan', 'All arti...",2021–2022 global supply chain crisis
377,COVID-19 pandemic in Bonaire,The COVID-19 pandemic in Bonaire is part of th...,https://en.wikipedia.org/wiki/COVID-19_pandemi...,"['2020 in Bonaire', '2021 in Bonaire', 'Articl...",2021–2022 global supply chain crisis


In [20]:
# wikidata_test.tail()

In [33]:
# wiki_data = wiki_data[~wiki_data['news'].str.contains('COVID-19')]

In [9]:
def clean_wiki_data(x):
    list_data = x.split('\n\n')
    final_list = []
    for sent in list_data:
        text = sent.split('\n')
        for tx in text:
            # print(tx)
            if len(tx.split()) > 5:
                # f_text = ''.join(tx)
                # print(f_text)
                final_list.append(tx)
                # break
    # final_text = ' '.join(final_list)
    return(final_list)

In [10]:
wiki_data['news'] = wiki_data['news'].apply(lambda x : clean_wiki_data(x))
wiki_data = wiki_data.reset_index(drop=True)

In [14]:
wiki_data['news'][0][0]

'A chip shortage, also referred to as semiconductor shortage or chip famine, is a phenomenon in the integrated circuit (chip) industry when demand for silicon chips outstrips supply.'

In [13]:
wiki_data.head()

,page,news,url,categories,topic
0,chip shortage,"[A chip shortage, also referred to as semicond...",https://en.wikipedia.org/wiki/Chip_shortage,"['All Wikipedia articles in need of updating',...",chip shortage
1,Broadcom Inc.,"[Broadcom Inc. is an American designer, develo...",https://en.wikipedia.org/wiki/Broadcom_Inc.,"['1961 establishments in California', '2009 in...",chip shortage
2,2011 Tōhoku earthquake and tsunami,[The 2011 Tōhoku earthquake and tsunami (Japan...,https://en.wikipedia.org/wiki/2011_T%C5%8Dhoku...,"['2011 Tōhoku earthquake and tsunami', '2011 d...",chip shortage
3,2020–present global chip shortage,[The 2020–present global chip shortage is an o...,https://en.wikipedia.org/wiki/2020%E2%80%93pre...,"['2020 in computing', '2020s economic history'...",chip shortage
4,Capital expenditure,[Capital expenditure or capital expense (capex...,https://en.wikipedia.org/wiki/Capital_expenditure,"['Accounting terminology', 'All articles needi...",chip shortage


In [15]:
def create_svo_ent_tkn_wiki(wiki_data):
    wk_doc_sub_v_sub=list() 
    wk_root_verbs=list()
    wk_token_entity=list()
    for j in range((wiki_data.shape[0])):
        data = wiki_data['news'][j]
        for sent in data:
            # sample_data_new = rem_title(sent)
            int_text=[x.translate(str.maketrans('','',punct)) for x in  sent.split()]
            # print(' '.join(int_text))
            doc=nlp(' '.join((int_text)))
            for tok in doc:
                if tok.dep_=='ROOT' and tok.pos_=='VERB':
                    wk_root_verbs.append(str(tok))
            # print(wk_root_verbs)

            # To solve coref issue
            doc_coref_resolved=nlp(doc._.coref_resolved)

            #Identifying entity types. Need some new code to add new entities
            for tok in doc_coref_resolved.ents:
                if tok.label_ not in reject_ent_list:
                    wk_token_entity.append([str(tok),tok.label_])
            # print(wk_token_entity)

            wk_SVO=[]
            #SVO using a custom code to handle active and passive voice
            for sent in doc_coref_resolved.sents:
                text = findSVOs(sent)
                wk_SVO.append(text)
            wk_doc_sub_v_sub.append([item for sublist in wk_SVO for item in sublist if item != []])
            # print(wk_doc_sub_v_sub)

                ## The below snippets appends needed columns from the news data with the subject object and verb tripples    
            wk_l_sub_v_sub=list()
            for i in range(len(wk_doc_sub_v_sub)):
                # new=list(sample_data.loc[i].values)[:4]

                for j in range(len(wk_doc_sub_v_sub[i])):
                    #df_needed_fields.append([new+[t for t in  doc_sub_v_sub[i][j]]])
                    wk_l_sub_v_sub.append([str(sub_obj) for sub_obj in  wk_doc_sub_v_sub[i][j]])
        # print(wk_l_sub_v_sub)
        # print(wk_token_entity)
    return wk_l_sub_v_sub, wk_token_entity

In [16]:
%%time
wiki_sub_vb_obj, wiki_token_entity = create_svo_ent_tkn_wiki(wiki_data)
df_doc_svb_wiki = pd.DataFrame(wiki_sub_vb_obj,columns=['subject','verb','object'])
df_tkn_ent_wiki = pd.DataFrame(wiki_token_entity,columns=['Token','Entity'])

Wall time: 48min 35s


In [17]:
df_tkn_ent_wiki.Entity.unique()

array(['NORP', 'PERSON', 'ORG', 'GPE', 'PRODUCT', 'LANGUAGE', 'EVENT'],
      dtype=object)

In [19]:
# df_doc_svb_wiki.to_csv('Data/sbvbob_wiki_lib2.csv',index=False)
# df_tkn_ent_wiki.to_csv('Data/tkn_ent_wiki_lib2.csv', index=False)

In [20]:
df_doc_svb_wiki = pd.read_csv('Data/sbvbob_wiki_lib2.csv')
df_tkn_ent_wiki = pd.read_csv('Data/tkn_ent_wiki_lib2.csv')

In [21]:
len(df_doc_svb_wiki)

73587

##### Wiki News Data Processing


In [22]:
# The below dataframe lowers the tokens in both the dataframe
df_tkn_ent_wiki['Token']=df_tkn_ent_wiki['Token'].str.lower()
df_tkn_ent_wiki=df_tkn_ent_wiki.drop_duplicates()
df_tkn_ent_wiki=df_tkn_ent_wiki.drop_duplicates(subset='Token', keep="first")
df_doc_svb_wiki['subject']=df_doc_svb_wiki['subject'].str.lower()
df_doc_svb_wiki['object']=df_doc_svb_wiki['object'].str.lower()

In [23]:
len(df_doc_svb_wiki)

73587

In [24]:
df_tkn_ent_wiki.head()

,Token,Entity
0,american,NORP
1,japanese,NORP
4,gateway,PERSON
5,intel,ORG
6,amd,ORG


In [25]:
#Creating the sub verb and object dataframe and joining the entity to find the subject and object entities
df_doc_svb_wiki=df_doc_svb_wiki.merge(df_tkn_ent_wiki,how='left', left_on='subject',right_on='Token')
df_doc_svb_wiki=df_doc_svb_wiki.merge(df_tkn_ent_wiki,how='left', left_on='object',right_on='Token')
print('1.',len(df_doc_svb_wiki))
#renaming and dropping unnecessary columns
df_doc_svb_wiki=df_doc_svb_wiki.rename(columns={"Entity_x":"Subject_Entity","Entity_y":"Object_Entity"})
df_doc_svb_wiki=df_doc_svb_wiki.drop(columns=['Token_x','Token_y'])
print('2.',len(df_doc_svb_wiki))
# Removing rows that does not have an entity defined and where subject and object are the same
df_doc_svb_wiki=df_doc_svb_wiki[df_doc_svb_wiki['Subject_Entity'].notnull() & df_doc_svb_wiki['Object_Entity'].notnull()]
df_doc_svb_wiki=df_doc_svb_wiki[df_doc_svb_wiki['subject'].notnull() & df_doc_svb_wiki['object'].notnull()]
df_doc_svb_wiki=df_doc_svb_wiki[df_doc_svb_wiki['subject']!=df_doc_svb_wiki['object']]
print('3.',len(df_doc_svb_wiki))

1. 73587
2. 73587
3. 1845


##### Stop word Removal

In [26]:

spacy_stop=list(nlp.Defaults.stop_words)
complete_stop_word=list(set(l_stopwords+spacy_stop))

df_doc_svb_wiki=df_doc_svb_wiki[~df_doc_svb_wiki['verb'].isin(complete_stop_word)]

data_verbs=df_doc_svb_wiki['verb'].to_list()
lemm_verbs=[]
j=0
for i in (data_verbs):
    doc_verb=nlp(i)
    verbs=[]
    for tok in doc_verb:
        verbs.append(tok.lemma_)
    verbs=' '.join(verbs)
    lemm_verbs.append(verbs)
    j=j+1
    if(j%10000==0):
        print(j)
df_doc_svb_wiki['lemm_verbs']=lemm_verbs
df_doc_svb_wiki=df_doc_svb_wiki[~df_doc_svb_wiki['lemm_verbs'].isin(complete_stop_word)]
print('4.',len(df_doc_svb_wiki))

4. 1644


#### Find word Synonyms

In [27]:
import spacy as spacy_nlp
from spacy_wordnet.wordnet_annotator import WordnetAnnotator
# spacy_nlp = spacy.load('en')
nlp_lemma_word_ann = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_lemma_word_ann.add_pipe(WordnetAnnotator(nlp_lemma_word_ann, name="spacy_wordnet"), after='tagger')

nlp_lemma_NO_word_annot = spacy.load('en_core_web_sm', disable=['parser', 'ner','WordnetAnnotator'])

lemm_verbs_no_stopwords=df_doc_svb_wiki['lemm_verbs'].to_list()
#lemm_verbs_no_stopwords=[i.replace('! ', '') for i in lemm_verbs_no_stopwords]
sorted_lemma_count=sorted(Counter(lemm_verbs_no_stopwords).items(), key=operator.itemgetter(1),reverse=True) 
 
#Get just the verbs from the previous list and ignore the verb count
lemm_verbs_synonym=[a_tuple[0] for a_tuple in sorted_lemma_count]
all_synonyms = []
prev_all_synonyms=[]
synn_list=[]
#Find the synonyms of the top verbs and replace that verb if any synonym is found
for i in range(len(sorted_lemma_count)):
     
    #all_synonyms=[item for sublist in all_synonyms for item in sublist]
     
    if sorted_lemma_count[i][0] not in prev_all_synonyms:#Check is needed so that the same word does not get updated
        #print(sorted_lemma_count[i][0])
        if (len(sorted_lemma_count[i][0].split())==1):
            for word in  nlp_lemma_word_ann(sorted_lemma_count[i][0]):
                synonyms = []
                for sysnet in word._.wordnet.synsets():

                    for l in sysnet.lemmas():
                        synonyms.append(l.name())
                        all_synonyms.append(l.name())
            synonyms=list(set(synonyms))
            #print(synonyms)
            all_synonyms=list(set(all_synonyms))
            synn_list.append((sorted_lemma_count[i][0],synonyms))

            for n,j in enumerate(lemm_verbs_no_stopwords):
                #print(j,prev_all_synonyms)
                if j in synonyms and j not in prev_all_synonyms : #and lemm_verbs_no_stopwords[n]!=sorted_lemma_count[i][0]
                    #print(sorted_lemma_count[i][0],j)
                    if (len(j.split())==1):
                    #print(j,sorted_lemma_count[i][0])

                    #idx=lemm_verbs_no_stopwords.index(j)
                    #print(j ,sorted_lemma_count[i][0])
                        #print(j,'before',lemm_verbs_no_stopwords[n],'after',sorted_lemma_count[i][0])
                        lemm_verbs_no_stopwords[n]=sorted_lemma_count[i][0]
                        #print('after',lemm_verbs_no_stopwords[n])
              
            prev_all_synonyms=all_synonyms
    if(i%1000==0):
        print(i)
df_doc_svb_wiki['synn_verbs']=lemm_verbs_no_stopwords

0


In [50]:
# len(full_s_o_list)

##### Simialiarity Measure

In [28]:
%%time
# Creating subject and object list
subject_list=df_doc_svb_wiki['subject'].to_list()
subject_list=[item.lower() for item in subject_list]


object_list=df_doc_svb_wiki['object'].to_list()
object_list=[item.lower() for item in object_list]
 
full_s_o_list=subject_list+object_list


import py_stringmatching as sm
 
SW=sm.SmithWaterman()
GJ=sm.GeneralizedJaccard()
lev = sm.Levenshtein()
jac = sm.Jaccard()
ocf=sm.OverlapCoefficient()
 
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
 
from py_stringmatching.similarity_measure.partial_token_sort import PartialTokenSort
pr= PartialRatio()
pts= PartialTokenSort()


full_s_o_list_count=sorted(Counter(full_s_o_list).items(), key=operator.itemgetter(1),reverse=True) 

#Get just the subject and object from the previous list and ignore the  count
full_s_o_list_final=[a_tuple[0] for a_tuple in full_s_o_list_count] 


ws_tok_set= sm.WhitespaceTokenizer(return_set=True)
calc_score_temp=[]
all_sym=[]
sim_lis=[]
prev_all_sym=[]
  
for i in range(len(full_s_o_list_final)):
     
    if full_s_o_list_final[i]  not in all_sym: #Check is needed so that the same word does not get updated
        similarity_list = []
        calc_score_temp=[]
        for j in range(len(full_s_o_list_final)):
             
            # score=(2*(SW.get_raw_score( full_s_o_list_final[i],full_s_o_list_final[j]))/(len(full_s_o_list_final[i])+len(full_s_o_list_final[j])))
            score=jac.get_raw_score(ws_tok_set.tokenize(full_s_o_list_final[i]), ws_tok_set.tokenize(full_s_o_list_final[j]))
            
            calc_score_temp.append((full_s_o_list_final[i],full_s_o_list_final[j],score))
            
            if score > .5:
                all_sym.append(full_s_o_list_final[j])
                similarity_list.append(full_s_o_list_final[j])
        sim_lis.append((full_s_o_list_final[i],similarity_list))
        for n,k in enumerate(subject_list):
            if k in similarity_list and k not in prev_all_sym :
                 subject_list[n]=full_s_o_list_final[i]
        for n,k in enumerate(object_list):
            if k in similarity_list and k not in prev_all_sym :
                object_list[n]=full_s_o_list_final[i]
    if(i%1000==0):
        print(i)

0
1000
Wall time: 9.28 s


In [29]:
df_doc_svb_wiki['subject_sim']=subject_list
df_doc_svb_wiki['object_sim']=object_list

df_doc_sub_v_sub_wiki_final = df_doc_svb_wiki[df_doc_svb_wiki['subject_sim']!=df_doc_svb_wiki['object_sim']] 

In [30]:
df_doc_sub_v_sub_wiki_final['verb_source_type']='Wiki'

df_doc_sub_v_sub_wikigraph_out=df_doc_sub_v_sub_wiki_final[['subject_sim','Subject_Entity','object_sim','Object_Entity','synn_verbs','verb_source_type']]
df_doc_sub_v_sub_wikigraph_out=df_doc_sub_v_sub_wikigraph_out.rename(columns={'subject_sim':'subject','Subject_Entity':'subject_entity','object_sim':'object','Object_Entity':'object_entity','synn_verbs':'synn_verbs'})



C:\Users\bdivedi\Anaconda3\envs\nlp_proj\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_doc_sub_v_sub_wikigraph_out = df_doc_sub_v_sub_wikigraph_out[(df_doc_sub_v_sub_wikigraph_out['object'].str.len()>2) & (df_doc_sub_v_sub_wikigraph_out['subject'].str.len()>2)] 
df_doc_sub_v_sub_wikigraph_out = df_doc_sub_v_sub_wikigraph_out[(df_doc_sub_v_sub_wikigraph_out['object'].str.len()>2) & (df_doc_sub_v_sub_wikigraph_out['subject'].str.len()>2)] 

In [32]:
df_graph_wiki = df_doc_sub_v_sub_wikigraph_out[['subject','object','synn_verbs','subject_entity','object_entity','verb_source_type']]
df_graph_wiki.columns = [':START_ID',':END_ID',':TYPE','subject_entity','object_entity','source_type']

In [33]:
df_graph_wiki=df_graph_wiki[df_graph_wiki[':END_ID']!=df_graph_wiki[':TYPE']] 
df_graph_wiki = df_graph_wiki.drop_duplicates()
df_graph_wiki.reset_index(drop=True, inplace=True)
df_graph_wiki.to_csv('Data/wiki_total_1208.csv',index=False)
df_graph_wiki.head()

,:START_ID,:END_ID,:TYPE,subject_entity,object_entity,source_type
0,chips,resin,use,ORG,ORG,Wiki
1,avago technologies,infineon technologies,acquire,ORG,ORG,Wiki
2,avago technologies,broadcom corporation,buy,ORG,ORG,Wiki
3,qualcomm,broadcom,purchase,PERSON,ORG,Wiki
4,the european union,broadcom,stop,ORG,ORG,Wiki


In [34]:
# df_tkn1 = df_graph_news[[':START_ID','subject_entity']]
# df_tkn1.columns = ['name',':LABEL']
# df_tkn2 = df_graph_news[[':END_ID','object_entity']]
# df_tkn2.columns = ['name',':LABEL']
# df_tkn3 = df_graph_wiki[[':START_ID','subject_entity']]
# df_tkn3.columns = ['name',':LABEL']
# df_tkn4 = df_graph_wiki[[':END_ID','object_entity']]
# df_tkn4.columns = ['name',':LABEL']
# df_tkn_merged = pd.concat([df_tkn1,df_tkn2,df_tkn3,df_tkn4], ignore_index=True)
# df_tkn_merged = df_tkn_merged.drop_duplicates(subset=['name'])
# df_tkn_merged.reset_index(inplace = True)
# df_tkn_merged.reset_index(inplace = True)
# df_tkn_merged = df_tkn_merged.rename(columns={'level_0':'Id:ID'})
# df_tkn_merged['Id:ID'] = df_tkn_merged['name'].apply(lambda x : x.upper())
# df_tkn_merged = df_tkn_merged.drop('index',axis=1)
# df_tkn_merged.head()

In [35]:
def create_id(x):
    text = x.split()
    text = '_'.join(text)
    return text

In [106]:
# df_tkn_merged['Id:ID'] = df_tkn_merged['Id:ID'].apply(lambda x : create_id(x))

In [61]:
# df_tkn_merged.head()

In [108]:
# df_tkn_merged = df_tkn_merged.drop_duplicates(subset=['Id:ID'])
# df_tkn_merged = df_tkn_merged[(df_tkn_merged['Id:ID'].str.len()>2)] 
# df_tkn_merged[df_tkn_merged['Id:ID'] == 'SI']

In [109]:
# df_tkn_merged.to_csv('node_all_1207_2.csv',index = False)

In [36]:
# len(df_tkn_merged)